In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

# Do new simulations 

In [ ]:
from simulation import run_simulation

outfolder = 'results/{}/'.format('testing')

### Option 1: Reproduce results

#run_simulation(outfolder, outfolder=None)

### Option 2: Add new results to existing folder 

#run_simulation(outfolder, outfolder=outfolder)

### Option 3: Create completely new results or add new to existing folder.

parameters = {
    'key': 'testing', 
    'n_its': 10, 
    'time': 'undefined', 
    'type_missing': 'uniform',
    'values_missing': range(3),
    'n_samples': [10], #[6, 8, 10, 12, 14, 16],
    'n_complexity': [4], #[4, 5, 6], 
    'n_anchors': [6], 
    'sigmas_noise': [0],
}
__ = run_simulation(parameters, outfolder, solver="rightInverseOfConstraints")

In [ ]:
from plotting_tools import add_plot_decoration
from plotting_tools import add_colorbar

colormap = "YlGn"
size = (9, 5)

def get_indices(results_header, x_slice, y_slice, parameters):
    indices = [[0]]*len(results_header)
    x_idx = results_header.index(x_slice)
    indices[x_idx] = range(len(parameters[x_slice]))
    y_idx = results_header.index(y_slice)
    indices[y_idx] = range(len(parameters[y_slice]))
    return indices

def plot_indices(results, indices, parameters, key, ax=None):
    """ plot a 2D-slize of results. 
    
    NOTE: values are nan if never visited (white)
    
    :param x_slice: parameter to plot on x-axis (from results_header)
    :param y_slice: parameter to plot on y-axis (from results_header)
    :param key: key to be plotted (from keys of results)
    
    """
    grid = np.meshgrid(*indices)
    data = results[key]
    data = np.squeeze(data[grid])

    if ax is None:
        fig = plt.figure()
        fig.set_size_inches(*size)
        ax = plt.gca()
    else:
        fig = plt.gcf()

    im = ax.matshow(data, cmap=colormap, aspect='auto')
    add_colorbar(im=im, fig=fig, ax=ax)
    return fig, ax
        
def get_theoretical(results_header, parameters):
    from global_variables import DIM
    
    bound_theory = np.zeros([
        len(parameters['n_complexity']), 
        len(parameters['n_anchors']), 
        len(parameters['n_samples']), 
        len(parameters['sigmas_noise']), 
        len(parameters['values_missing'])])
    for c_idx, n_complexity in enumerate(parameters['n_complexity']):
        for a_idx, n_anchors in enumerate(parameters['n_anchors']):
            for p_idx, n_samples in enumerate(parameters['n_samples']):
                for noise_idx in range(len(parameters['sigmas_noise'])):
                    for m_idx in range(len(parameters['values_missing'])):

                        max_measurements = n_samples * n_anchors
                        if parameters['type_missing'] == 'n-closest':
                            num_measurements = n_samples * m_idx
                        elif parameters['type_missing'] == 'uniform':
                            num_measurements = max_measurements - m_idx

                        min_measurements = DIM * n_complexity + 2 * n_complexity
                        if (num_measurements >= min_measurements):
                            bound_theory[c_idx, a_idx, p_idx, noise_idx, m_idx] = 1.0
                        else:
                            bound_theory[c_idx, a_idx, p_idx, noise_idx, m_idx] = 0
    return bound_theory


from matplotlib.colors import hsv_to_rgb
def plot_all(results, results_theoretical, indices, parameters, key):
    grid = np.meshgrid(*indices)
    data = results[key]
    data = np.squeeze(data[grid])
    
    data_norm = data / np.max(data)
    print(np.min(data_norm), np.max(data_norm))
    
    data_theoretical = np.squeeze(results_theoretical[grid])

    image = np.zeros((*data.shape, 4), dtype=np.float32)
    
    image[:, :, 0] = data_norm
    
    # create shadow over theoretical bounds.
    image[data_theoretical == 0, 3] = 1.0
    image[data_theoretical > 0, 3] = 0.8
    
    # create green shade 
    #image[:, :, 3] = 1.0
    #image[data_theoretical > 0, 1] = 0.9

    #image_rgb = hsv_to_rgb(image)
    image_rgb=image
    
    fig = plt.figure()
    fig.set_size_inches(*size)
    ax = plt.gca()
    ax.imshow(image_rgb, aspect='auto')
    return fig, ax
                            
def plot_theoretical(bound_theory, indices):
    fig = plt.figure()
    fig.set_size_inches(*size)
    ax = plt.gca()
    grid = np.meshgrid(*indices)        
    im = ax.matshow(np.squeeze(bound_theory[grid]), cmap=colormap, aspect='auto')
    add_colorbar(im=im, fig=fig, ax=ax)
    add_plot_decoration(ax, x_slice, y_slice, parameters)
    ax.set_title('Theoretical bound')
    return fig, ax

In [ ]:
from simulation import read_results
from json_io import read_json, write_json

## noiseless test, different models.
#['n-closest', {'n':4}]], 

sigmas = [0]
n_anchors = [5]
n_samples = [20]
max_measurements = max(n_anchors) * max(n_samples)
parameters = {
    'key': 'uniform', 
    'time': 'undefined', # will be overwritten. 
    'type_missing': 'uniform',
    'values_missing': range(max_measurements),
    'n_its': 10, 
    'n_samples': n_samples,
    'n_complexity': range(4, 8), 
    'n_anchors': n_anchors, 
    'sigmas_noise': sigmas,
}

outfolder = 'results/{}/'.format(parameters['key'])
results1, results1_header = run_simulation(
    parameters, outfolder, solver="rightInverseOfConstraints")

params2 = read_json(outfolder + 'parameters.json')

In [ ]:
results1, results1_header = read_results(outfolder + 'result_')
parameters = read_params(outfolder + 'parameters.json')

results_theory = get_theoretical(results1_header, parameters)

# 2D Plot
# Choose which parameters to plot - the values are indexes to the previously given parameters
x_slice = 'values_missing'
y_slice = 'n_complexity'

indices = get_indices(results1_header, x_slice, y_slice, parameters)
fix, ax  = plot_indices(results1, indices, parameters, 'successes')
add_plot_decoration(ax, x_slice, y_slice, parameters)

fig, ax = plot_theoretical(results_theory, indices)
add_plot_decoration(ax, x_slice, y_slice, parameters)
    
fig, ax = plot_all(results1, results_theory, indices, parameters, 'errors')
add_plot_decoration(ax, x_slice, y_slice, parameters)

In [ ]:
#['n-closest', {'n':4}]], 
sigmas = [0]
n_anchors = [4]
n_samples = range(20, 40)
parameters2 = {
    'key': 'testing', 
    'time': 'undefined', # will be overwritten. 
    'type_missing': 'n-closest',
    'values_missing': [1, 2, 3],
    'n_its': 10, 
    'n_samples': n_samples,
    'n_complexity': range(3, 15), 
    'n_anchors': n_anchors, 
    'sigmas_noise': sigmas,
}
#outfolder = 'results/{}/'.format(parameters['key'])
outfolder = None
results2, results2_header = run_simulation(
    parameters, outfolder, solver="rightInverseOfConstraints")

#results = read_results(outfolder + 'result_')
#p = read_params(outfolder + 'parameters.json')

In [ ]:
x_slice = 'n_samples'
y_slice = 'n_complexity'

results2_theory = get_theoretical(results2_header, parameters2)

for i in range(3):
    indices = get_indices(results2_header, x_slice, y_slice, parameters2)
    indices[results2_header.index('values_missing')] = [i]
    fix, ax  = plot_indices(results2, indices, parameters2, 'errors')
    add_plot_decoration(ax, x_slice, y_slice, parameters2)

    fig, ax = plot_theoretical(results2_theory, indices)
    add_plot_decoration(ax, x_slice, y_slice, parameters2)

    fig, ax = plot_all(results2, results2_theory, indices, parameters2, 'errors')
    add_plot_decoration(ax, x_slice, y_slice, parameters2)